<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #11: Twitter
`Fecha de entrega: Mayo 20, 2021 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [1]:
import pandas as pd
import tweepy
import re 
import regex
import emoji
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [2]:
api_key = "N1-Cr3a-Qu3-lE-D3jar3-La5"
api_secret_key = "Cr3D3nc1al3s-"
access_token = "Y-Much0-m3N0s-L0s-C0d1G0s"
access_token_secret =  "D3-Acc355O"

###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

❌ No escoger un tema de los que hicimos en clase (día de las madres, Giro de Italia, #EraGoldeYepes)

In [14]:
# Autenticar a través de la API 
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [15]:
tweets = tweepy.Cursor(api.search,
                       q=['#XboxGamePassUltimate -RT'],
                       lang='es',
                       # geocode='4.7110,-74.0721,100km', #Filtra por localizacion 
                       tweet_mode='extended',# Para que llegue el texto completo
                       include_rts=False, # Para evitar que sea re tuits
                       ).items(100)

In [16]:
# Convirte la informacion en una lista
tuits = [tweet for tweet in tweets]

In [18]:
tuits[0].full_text

'Ya quisiera poder llegar a casa para seguir dándole al Hollow Knight.\nSi aun no lo has probado, dale una oportunidad, no es un videojuego nuevo, pero sin duda alguna que es un gran videojuego. #xbox #xboxgamepass #XboxGamePassUltimate #xboxseriesX #HollowKnight #metroidvania https://t.co/YhPVyqstcc'

In [20]:
tweet = []

for tuit in tuits: 
    temp = []
    temp.append(tuit.full_text)
    temp.append(tuit.created_at)
    temp.append(tuit.id)
    temp.append(f"https://twitter.com/i/web/stuatus/{tuit.id}")
    
    tweet.append(temp)
    
    
data = pd.DataFrame(tweet)
data.columns = ['tweets', 'date', 'id', 'link']
data.head()

,tweets,date,id,link
0,Ya quisiera poder llegar a casa para seguir dá...,2021-05-19 23:54:26,1395165984869212160,https://twitter.com/i/web/stuatus/139516598486...
1,Cometí el error de principiante de Game Pass! ...,2021-05-19 22:33:02,1395145498969919492,https://twitter.com/i/web/stuatus/139514549896...
2,Quedan pocos días de mayo y muchos títulos nue...,2021-05-19 19:54:13,1395105533108097025,https://twitter.com/i/web/stuatus/139510553310...
3,¿ #XboxGamePassUltimate en #NintendoSwitch ?...,2021-05-19 19:51:23,1395104819967311875,https://twitter.com/i/web/stuatus/139510481996...
4,Amigo: Ven\n\nNosotros: No podemos\n\nAmigo: p...,2021-05-19 19:30:10,1395099478579892227,https://twitter.com/i/web/stuatus/139509947857...


In [21]:
pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweets']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
    
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag, "")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention, "")
        
    return text, links, hashtags, mentions

In [22]:
#Obtiene el texto limpio, acortamos la url, obtenemos los hashtags y las cuentas mencionadas
data[['texto_limpio', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')
data

,tweets,date,id,link,texto_limpio,links,hashtags,mentions
0,Ya quisiera poder llegar a casa para seguir dá...,2021-05-19 23:54:26,1395165984869212160,https://twitter.com/i/web/stuatus/139516598486...,Ya quisiera poder llegar a casa para seguir dá...,[https://t.co/YhPVyqstcc],"[#xbox, #xboxgamepass, #XboxGamePassUltimate, ...",[]
1,Cometí el error de principiante de Game Pass! ...,2021-05-19 22:33:02,1395145498969919492,https://twitter.com/i/web/stuatus/139514549896...,Cometí el error de principiante de Game Pass! ...,[],"[#XboxSeriesS, #XboxGamePassUltimate]",[]
2,Quedan pocos días de mayo y muchos títulos nue...,2021-05-19 19:54:13,1395105533108097025,https://twitter.com/i/web/stuatus/139510553310...,Quedan pocos días de mayo y muchos títulos nue...,"[https://t.co/BWnRMZiSYt, https://t.co/LGzgWYa...","[#XboxGamePassUltimate, #Xbox20, #XboxSeriesX,...",[@YouTube]
3,¿ #XboxGamePassUltimate en #NintendoSwitch ?...,2021-05-19 19:51:23,1395104819967311875,https://twitter.com/i/web/stuatus/139510481996...,¿ en ? El juicio de Epic contra Apple y un...,[https://t.co/CZopNvhq43],"[#XboxGamePassUltimate, #NintendoSwitch]",[]
4,Amigo: Ven\n\nNosotros: No podemos\n\nAmigo: p...,2021-05-19 19:30:10,1395099478579892227,https://twitter.com/i/web/stuatus/139509947857...,Amigo: Ven\n\nNosotros: No podemos\n\nAmigo: p...,[https://t.co/QM950LaKkz],[#XboxGamePassUltimate],[]
...,...,...,...,...,...,...,...,...
95,Pero que cojones de juego...?\n\n#XboxSeriesX ...,2021-05-13 20:11:06,1392935452466982913,https://twitter.com/i/web/stuatus/139293545246...,Pero que cojones de juego...?\n\n,[https://t.co/0vmmhZSy0W],"[#XboxSeriesX, #XboxGamePassUltimate, #Psychon...",[]
96,Este año es el 20 aniversario de Xbox y así se...,2021-05-13 20:05:00,1392933918630813700,https://twitter.com/i/web/stuatus/139293391863...,Este año es el 20 aniversario de Xbox y así se...,[https://t.co/aaTOApfT1Y],"[#XboxGamePassUltimate, #Xbox20]",[]
97,3) 3 Meses de Xbox Game Pass Ultimate por solo...,2021-05-13 17:47:28,1392899308672987142,https://twitter.com/i/web/stuatus/139289930867...,3) 3 Meses de Xbox Game Pass Ultimate por solo...,[https://t.co/9TF6yIXKHp],"[#XboxGamePass, #XboxGamePassUltimate, #Micros...",[]
98,Así se vería #XboxGamePassUltimate si se hubie...,2021-05-13 17:44:34,1392898578436268033,https://twitter.com/i/web/stuatus/139289857843...,Así se vería si se hubiera lanzado hace 20 añ...,[https://t.co/L9WsrOXmUH],[#XboxGamePassUltimate],[]


In [26]:
# Funcion que obtiene los emojis en el texto
def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)

    return emoji_list
  

In [29]:
data['emojis'] = data['texto_limpio'].apply(lambda text: get_emojis(text))
data.head()

,tweets,date,id,link,texto_limpio,links,hashtags,mentions,emojis
0,Ya quisiera poder llegar a casa para seguir dá...,2021-05-19 23:54:26,1395165984869212160,https://twitter.com/i/web/stuatus/139516598486...,Ya quisiera poder llegar a casa para seguir dá...,[https://t.co/YhPVyqstcc],"[#xbox, #xboxgamepass, #XboxGamePassUltimate, ...",[],[]
1,Cometí el error de principiante de Game Pass! ...,2021-05-19 22:33:02,1395145498969919492,https://twitter.com/i/web/stuatus/139514549896...,Cometí el error de principiante de Game Pass! ...,[],"[#XboxSeriesS, #XboxGamePassUltimate]",[],"[🥲, 🥲, 🥲]"
2,Quedan pocos días de mayo y muchos títulos nue...,2021-05-19 19:54:13,1395105533108097025,https://twitter.com/i/web/stuatus/139510553310...,Quedan pocos días de mayo y muchos títulos nue...,"[https://t.co/BWnRMZiSYt, https://t.co/LGzgWYa...","[#XboxGamePassUltimate, #Xbox20, #XboxSeriesX,...",[@YouTube],[]
3,¿ #XboxGamePassUltimate en #NintendoSwitch ?...,2021-05-19 19:51:23,1395104819967311875,https://twitter.com/i/web/stuatus/139510481996...,¿ en ? El juicio de Epic contra Apple y un...,[https://t.co/CZopNvhq43],"[#XboxGamePassUltimate, #NintendoSwitch]",[],[]
4,Amigo: Ven\n\nNosotros: No podemos\n\nAmigo: p...,2021-05-19 19:30:10,1395099478579892227,https://twitter.com/i/web/stuatus/139509947857...,Amigo: Ven\n\nNosotros: No podemos\n\nAmigo: p...,[https://t.co/QM950LaKkz],[#XboxGamePassUltimate],[],[]


###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

In [32]:
data['hora'] = data['date'].dt.floor('T').dt.time
temp = pd.DataFrame(data.hora.value_counts()).reset_index()
temp.columns = ['hora', 'cnt']
temp = temp.sort_values('hora')

In [33]:
trace = go.Scatter(x=temp.hora.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.hora.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Horas',
                   line={'color': 'blue'})

layout = go.Layout(title="Número de tuits por hora para #XboxGamePassUltimate")

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)